tensorboard --logdir="C:\Users\USER\Desktop\Git\capstone\Capstone\result\InceptionResNet_original_data_299_1000_0.0003_128\tensorboard\train"

In [ ]:
import numpy as np

def compute_embeddings(model, generator):
    embeddings = []
    labels = []
    for i in range(len(generator)):
        batch_x, _ = generator[i]
        anchor_embeddings, positive_embeddings, negative_embeddings = model.predict_on_batch(batch_x)
        embeddings.append(np.concatenate([anchor_embeddings, positive_embeddings, negative_embeddings], axis=0))
        labels.extend(generator.labels[generator.indices[i * generator.batch_size: (i + 1) * generator.batch_size]])
    embeddings = np.vstack(embeddings)
    labels = np.array(labels)
    return embeddings, labels

# Compute embeddings for the training set
train_embeddings, train_labels = compute_embeddings(model, train_triplet_gen)

# For evaluation, you would typically compare these embeddings using a metric like cosine similarity or Euclidean distance.
# For simplicity, here we'll just print the shape of the embeddings.
print('Train embeddings shape:', train_embeddings.shape)
print('Train labels shape:', train_labels.shape)


In [52]:
import tensorflow as tf

model = tf.keras.applications.EfficientNetB0(
	# input_shape=(299,299,3), 
	include_top=False, 
)
model.summary()

16719872/16705208 [==============================] - 1s 0us/step
Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling (Rescaling)          (None, None, None,   0           ['input_2[0][0]']                
                                3)                                                                
                                                                                                  
 normalization (Normalization)  (None, None, None,   7           ['rescaling[0][0]']              
                    

In [53]:
import tensorflow as tf

model = tf.keras.applications.EfficientNetB0(
	# input_shape=(299,299,3), 
	include_top=True, 
)
model.summary()

21848064/21834768 [==============================] - 1s 0us/step
Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                    

In [1]:
import os

WEIGHT_PATH = '../result/InceptionResNet_cropped_data_299_100_0.0003_256/finetune_checkpoint/checkpoint-029-0.772459-1.208163-0.740569-0.563585.weights.h5'

# Check if the file exists
if os.path.exists(WEIGHT_PATH):
    print(f"The file {WEIGHT_PATH} exists.")
else:
    print(f"The file {WEIGHT_PATH} does not exist.")


The file ../result/InceptionResNet_cropped_data_299_100_0.0003_256/finetune_checkpoint/checkpoint-029-0.772459-1.208163-0.740569-0.563585.weights.h5 exists.
